In [8]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2

loaded_model = load_model('faсe_recog50epochs.h5')
face_color = (0,0,255)
thickness = 2

In [9]:
classes = {0:'Akhmedov Sanzhar',
 1:'Alikhan Zhamankhan',
 2:'Baibatchayeva Anar',
 3:'Dalabay Abylay',
 4:'Daniyar Raiymbekov',
 5:'Duisenov Alikhan',
 6:'Kenzhebay Askar',
 7:'Kyran Yerkin',
 8:'Maxat Ualikhan',
 9:'Naryshov Ernar',
 10:'Rysmambetov Nurtugan',
 11:'Sagynbek Nurbolat',
 12:'Yerkin Yernar',
 13:'Zhetessov Nur',
 14:'Zhumabekov Magjan'}

In [10]:
face_cascade = cv2.CascadeClassifier('../DATA/DATA/haarcascades/haarcascade_frontalface_default.xml')
face_cascade

<CascadeClassifier 000002864615E2F0>

In [11]:
face_cascade = cv2.CascadeClassifier('../DATA/DATA/haarcascades/haarcascade_frontalface_default.xml')

def detect(img):    
    face_img = img.copy()
    face_gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    face_rects = face_cascade.detectMultiScale(face_gray, 1.3, 5)
    for (x,y,w,h) in face_rects:
        cv2.rectangle(face_img, (x,y), (x+w,y+h), face_color,thickness)  
        res = cv2.resize(face_img, (150, 150))
        res = image.img_to_array(res)
        res = np.expand_dims(res, axis=0)
        res = res/255
        prediction_prob = loaded_model.predict(res)
        max_res = max(prediction_prob[0])
        max_text = str(int(max_res*100))+'%'
        i = np.where(prediction_prob[0] == max_res)
        cv2.putText(face_img, classes[int(i[0])], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, face_color, 1)
        cv2.putText(face_img, max_text, (x+10, y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, face_color, 1)
    return face_img

In [16]:
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#writer = cv2.VideoWriter('project4_result.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))
cv2.namedWindow("video")
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    canvas = frame.copy()
    cv2.imshow('video', detect(canvas)) 
    #writer.write(detect(canvas))
    
    key = cv2.waitKey(1)
    if key == ord('q'): break

cap.release()
#writer.release()
cv2.destroyAllWindows()